In [ ]:
import os
import math
from ttictoc import tic, toc
import csv

print(os.getcwd())

In [ ]:
epochs0 = 20

%run utils/dataset_tools.ipynb
%run utils/network_tools.ipynb
%run utils/training_tools.ipynb

# Define speeds (m/s) corresponding to 0-40 Hz settings in wind tunnel
### (Just for record-keeping)
hzArray = np.array((0, 5, 10, 15, 20, 25, 30, 35, 40))
speedArray = np.array((0.00, 1.26363735, 1.58562983, 2.07066356, 2.571993, 3.18291372, 3.75322345, 4.33626595, 4.91413509))

print(torch.cuda.is_available())

In [ ]:
'''
Necessary Parameters (derived params double-indented)

--- trig value (1, 2)
--- --- Dataset (WindAng, WindAngTrig) [Done]
--- --- loss fn (customLoss, L1/L2?)

--- testPath0
--- --- writePath [Done]

--- verboseFlag

--- NN (array(1,2,5))
--- --- nFilt [Done]

--- GEOMVAL (arange(3,7))
--- --- nGeom [Done]

--- nSeed (5)
'''

trigValue = 1

if trigValue==1:
    loss_fn = customLoss()
elif trigValue==2:
    loss_fn = nn.MSELoss() # nn.L1Loss()
else:
    raise ValueError('Invalid value of trigValue; trigValue must be in {1, 2}.')
    
testPath0 = 'data_agg/test'

verboseFlag=False

NN = np.array([1, 2, 5])
GEOMVAL = np.arange(5, 7)
nSeed = 5

nFilt = len(NN)
nGeom = len(GEOMVAL)

In [ ]:
tic()
for jj in range(len(GEOMVAL)):
    # Sort out geometryVal and geomPath from jj
    geometryVal = GEOMVAL[jj]
    if geometryVal == 3:
        geomPath='tri/'
    elif geometryVal == 4:
        geomPath='squ/'
    elif geometryVal == 5:
        geomPath='pent/'
    elif geometryVal == 6:
        geomPath='hex/'
    else:
        raise ValueError('Geometry must be in {3, 4, 5, 6}')
        
    for ii in range(len(NN)):
        # Reset bestPerformance to pick out best set for each configuration
        bestPerformance = 100.0
        # Create N from ii
        N = NN[ii]
        
        for kk in range(nSeed):
            # Initialize the seed
            np.random.seed(kk*12345 + 31415*N*(geometryVal**3))
            
            # Make all necessary data using make*Dataset function
            training_data, testing_data, epochs, trainPath, trainLabelPath, testPath, testLabelPath = makeAngleDataset(trigValue=1, geomPath=geomPath, N=N, testPath0=testPath0, epochs0=epochs0)
            
            # Define the writePath for test errors, etc
            writePath = testPath+geomPath
            
            # Make training and testing dataloaders
            # Initialize model and optimizer params
            train_dataloader = DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
            test_dataloader = DataLoader(testing_data, batch_size=batch_size_test, shuffle=True)
            test_dataloader2 = DataLoader(testing_data, batch_size=1, shuffle=True)
            model = AngleNet(geom=geometryVal)
            opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
            
            print(f"Epoch {0}\n-------------------------------")
            avg_error = test_loop(test_dataloader, model, 0, kk, loss_fn, lastLoop=False, writePath=writePath)
            
            for t in range(1, epochs+1):
                print(f"Epoch {t}\n-------------------------------")
                train_loop(train_dataloader, model, opt, t, kk, loss_fn, verbose=verboseFlag, writePath=writePath)
                print()
                # if (float(t+1)/float(epochs) >= k or (t==(epochs-1))):
                if t < epochs:
                    avg_error = test_loop(test_dataloader, model, t, kk, loss_fn, lastLoop=False, writePath=writePath)
                else: 
                    # avg_error, Z = test_loop(test_dataloader, model, loss_fn, lastLoop=(t==(epochs-1)))
                    avg_error, Z = test_loop(test_dataloader2, model, t, kk, loss_fn, lastLoop=True, writePath=writePath)
            
            if avg_error < bestPerformance:
                bestPerformance = avg_error
                PATH = 'SavedModels/Angle/N'+str(N)+'_G'+str(geometryVal)+'_best.tar'
                torch.save({
                        'epoch': epochs,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': opt.state_dict(),
                        'loss': avg_error*180.0/math.pi,
                        }, PATH)
            # Print that we have finished training
            print(f"Finished seed: {kk+1:>2d} of "+str(nSeed)+f", on geometry {jj+1:>2d} of "+str(len(GEOMVAL))+f", on filtering setting {ii+1:>2d} of "+str(len(NN))+" \n")


# Output time elapsed in seconds            
dT = toc()
dT = np.round_(dT)
dT2 = dT % 3600
dT3 = dT2 % 60
print(f"Elapsed time is {dT} seconds")
print(f"This is equivalent to {dT // 3600} hours, "+f"{dT2 // 60} minutes, and "+f"{dT3} seconds")
